In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

2023-10-04 18:48:15.011666: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-10-04 18:48:16.327431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


True

2023-10-04 18:48:16.343381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 18:48:16.343599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 18:48:16.408330: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
train = pd.read_csv('./jigsaw-toxic-comment-train.csv')
valid = pd.read_csv('./validation.csv')
test = pd.read_csv('./test.csv')

In [4]:
train.drop(['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],axis=1, inplace=True)

In [5]:
train.shape

(223549, 3)

In [6]:
train['comment_text'].apply(lambda x: len(str(x).split())).max()

2321

In [7]:
from sklearn.metrics import roc_curve, auc

In [8]:
def roc_auc(predictions, target):
    fpr, tpr, thresholds = roc_curve(target, predictions)
    roc_auc = auc(fpr, tpr)
    return roc_auc

In [9]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train.comment_text.values, train.toxic.values,
    stratify = train.toxic.values,
    random_state = 42,
    test_size = 0.2, shuffle = True
)

In [10]:
from keras.preprocessing import text
from keras.utils import pad_sequences

token = text.Tokenizer(num_words=None)
max_len = 2500

token.fit_on_texts(list(x_train) + list(x_valid))
x_train_seq = token.texts_to_sequences(x_train)
x_valid_seq = token.texts_to_sequences(x_valid)

x_train_pad = pad_sequences(x_train_seq, maxlen=max_len)
x_valid_pad = pad_sequences(x_valid_seq, maxlen=max_len)

word_index = token.word_index

In [11]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding

In [12]:
model = Sequential()
model.add(Embedding(len(word_index)+1, 300, input_length=max_len))
model.add(SimpleRNN(100, dropout=0.3)) ## I would use recurrent dropout, but I don't want to lose timesteps 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2500, 300)         90077400  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 90,117,601
Trainable params: 90,117,601
Non-trainable params: 0
_________________________________________________________________


2023-10-04 18:48:28.806973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 18:48:28.807196: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-04 18:48:28.807368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [13]:
model.fit(x_train_pad, y_train, epochs=1, batch_size=128)

2023-10-04 18:48:30.528491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-04 18:48:30.944574: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fea4b811230 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-04 18:48:30.944609: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-10-04 18:48:30.948441: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-04 18:48:30.957695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-10-04 18:48:31.031680: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

1398/1398 [==============================] - 1357s 970ms/step - loss: 0.1852 - accuracy: 0.9366


In [14]:
scores = model.predict(x_valid_pad)

1398/1398 [==============================] - 126s 90ms/step


In [15]:
print(f"AUC: {roc_auc(scores, y_valid)}")

AUC: 0.9520275493814067


In [16]:
## We can do better!!
## Maybe LSTMs or BERT..?